<img align="left" style="padding-right:30px;" src="./Images/logo_UNSW.svg" height="100"><img align="left" style="padding-right:30px;" src="./Images/Green_RGB.png" height="90"><img align="left" style="padding-right:30px;" src="./Images/image004.png" height="100">



[![DOI](https://zenodo.org/badge/940091341.svg)](https://doi.org/10.5281/zenodo.14939868) <a href="https://www.globh2e.org.au/"><img src="https://img.shields.io/badge/ARC:Funding%20number-IC200100023-blue.svg"/></a>

# **<div style="text-align: left; font-size: 30px;"> Pathways to global hydrogen production within planetary boundaries**</div>
**<div style="text-align: left; font-size: 25px;"> Life cycle optimisation - Tutorial**</div>

<div style="text-align: left; font-size: 16px;">Michaël Lejeune<sup>a,b</sup>, Sami Kara<sup>a,b</sup>, Michael Zwicky Hauschild<sup>c,d</sup>, Sareh Sharabifarahni<sup>a</sup>, Rahman Daiyan<sup>b,e</sup></div><br>

<div style="text-align: left; font-size: 13px;"><sup>a</sup>Sustainability in Manufacturing and Life Cycle Engineering Research Group, School of Mechanical and Manufacturing Engineering, the University of New South Wales, 2052, Sydney, Australia</div>

<div style="text-align: left; font-size: 13px;">
<sup>b</sup>Australian Research Council Training Centre for the Global Hydrogen Economy (GlobH2e), the University of New South Wales, 2052, Sydney, Australia</div>

<div style="text-align: left; font-size: 13px;">
<sup>c</sup>Centre for Absolute Sustainability, Technical University of Denmark, Kgs, Lyngby, Denmark</div>

<div style="text-align: left; font-size: 13px;">
<sup>d</sup>Section for Quantitative Sustainability Assessment (QSA), Department of Environmental and Resource Engineering, Technical University of Denmark, Kgs, Lyngby, Denmark</div>

<div style="text-align: left; font-size: 13px;">
<sup>e</sup>School of Minerals and Energy Engineering, The University of New South Wales, Sydney 2052, Australia</div><br>


> <span style="color:rgba(22, 210, 69, 1); font-weight: bold;">Target audience</span><br>
>- Life cycle assessment practioners/researchers
>- Absolute environmental sustainability practioners/researchers

> <span style="color:rgba(191, 200, 30, 1); font-weight: bold;">Prerequisites</span><br>
>- Background in life cycle assessment
>- Brightway
>- Activity browser (with ScenarioLink installed)
>- Julia with JuMP.jl
>- CPLEX optimisation solver
>- IDE (e.g., VS code)

# <span style="color:rgba(0, 0, 0, 1); font-weight: bold;">Contents</span><br>
>1. Background information on computational LCA and optimisation
>2. Installation of planetary boundaries characterisation factors
>3. Modelling aggregated inventories on Activity - browser
>4. pre-optimisation formating on Excel
>5. Implementation of the optimisaiton model in Julia using JuMP.jl



# **Case study**

<!-- > <span style="color:rgba(184, 193, 29, 1); font-weight: bold;">Simple optimisation example</span><br>
>- Optimisation of electrical mix for hydrogen production via PEM electrolysis
>- Using activity browser, excel and Julia (a python version is also possible)
>- Integration of prospective LCA data in the optimisation.
>- Optional - Cost and material tracking -->


> <span style="color:rgba(26, 190, 67, 1); font-weight: bold;">Goal  & Scope definition</span><br>
>- Goal: Minimising the effective planetary footprint of hydrogen production via PEM electrolysis.
>- LCA type: prospective attributional LCA (note that the appraoch can also be interpreted as consequential)
>- Functional unit: 225 MtH₂/year (2050) (global annual production of H₂ via water electrolysis)
>- Scope: Cradle to gate (electricity production + PEM electrolysis)
>- Database: ecoinvent 3.9.1 (cut-off) - updated with premise - SSP1-PkBudg500
<div style="text-align:center;">
  <img src="./Images//PEM_unit1.svg">
</div>


### **Initialisation**

In [78]:
using JuMP, CPLEX,Distributions,lce,DataFrames,LinearAlgebra,XLSX
using PyPlot
using SparseArrays
using JLD2
# initProject("Natcoms")
⊘ = ./;


Data import

In [10]:
aSOS = XLSX.readtable("./data/Fig2c.xlsx", "allocated SOS") |> DataFrame
𝚪ᵦ=XLSX.readtable("./data/interactions_matrices.xlsx", "𝚪ᵦ") |> DataFrame
SOS = XLSX.readtable("./data/SOS.xlsx", "Planetary boundaries") |> DataFrame


Row,Control variable,Pre-industrial value,Planetary boundary,SOS,High risk,High risk space,Units
,Any,Any,Any,Any,Any,Any,Any
1,Climate change-Energy imbalance,0.0,1.0,1.0,1.5,1.5,W/m2
2,Climate change-CO2 Concentration,278.0,350.0,72.0,450.0,172.0,ppm
3,Ocean acidification-Carbonate ion concentration,3.44,2.752,0.688,2.408,1.032,Ωarag
4,Atmospheric aerosol loading-Aerosol Optical Depth (AOD),0.03,0.1,0.07,0.25,0.22,Aerosol optical depth
5,Freshwater use-Global,0.0,4000.0,4000.0,6000.0,6000.0,km3
6,Biogeochemical flows-P,0.0,11.0,11.0,100.0,100.0,TgP
7,Biogeochemical flows-N,0.0,62.0,62.0,82.0,82.0,TgN
8,Stratospheric ozone depletion-Stratospheric O3 concentration,290.0,276.0,14.0,261.0,29.0,Dobson units
9,Land-system change-Global,100.0,75.0,25.0,54.0,46.0,%


Safe operaring space 

In [49]:
categories=SOS[:,1]
Δ𝐗ᴾᴮ=SOS[:,4]
units=SOS[:,end]


10-element Vector{Any}:
 "W/m2"
 "ppm"
 "Ωarag"
 "Aerosol optical depth"
 "km3"
 "TgP"
 "TgN"
 "Dobson units"
 "%"
 "%"

Allocation factor

In [51]:
α_2050=aSOS[end,4] #%SOS/GtH2 (median value)


1.5974315890692654

We want to express it in an allocation factor per kg of H2 produced.

In [79]:
ω_2050°=α_2050./100*Δ𝐗ᴾᴮ # allocation factor per GtH2
𝛚=ω_2050° * 1e-12 # allocation factor per kgH2


10-element Vector{Float64}:
 1.5974315890692653e-14
 1.1501507441298711e-12
 1.099032933279655e-14
 1.118202112348486e-15
 6.389726356277062e-11
 1.757174747976192e-13
 9.904075852229445e-13
 2.2364042246969718e-13
 3.9935789726731633e-13
 1.5974315890692653e-13

Now we compute the interaction matrix, we consider biophysical interactions only.

In [55]:
𝚪ᵦ


Row,Climate change Energy imbalance,Climate change CO2 Concentration,Ocean acidification,Atmospheric aerosol loading,Freshwater use,Biogeochemical flows-P,Biogeochemical flows-N,Stratospheric ozone depletion,Land-system change,Biosphere Integrity
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1.22241,0,-0.0100732,0.0232257,-0.111239,0.232257,0.232257,-0.0710218,0.122241,0.38386
2,0,1.22241,-0.0100732,0.0232257,-0.111239,0.232257,0.232257,-0.0710218,0.122241,0.38386
3,0.325108,0,1.06115,0.00617706,-0.0295849,0.0617706,0.0617706,-0.0188888,0.0325108,0.4567
4,-0.684548,0,0.00564097,0.986994,0.0622939,-0.130064,-0.130064,0.0397722,-0.0684548,-0.214962
5,0.220814,0,0.0407336,0.00419546,0.979906,0.0419546,0.0419546,-0.0128293,0.0220814,0.416858
6,0.206199,0,0.0138327,0.103918,-0.0187641,1.03918,1.03918,-0.00198018,0.0206199,0.426595
7,0.206199,0,0.0138327,0.103918,-0.0187641,1.03918,1.03918,-0.00198018,0.0206199,0.426595
8,-0.134465,0,0.00110805,-0.00255483,0.0122363,-0.0255483,-0.0255483,1.00781,-0.0134465,-0.0422246
9,0.428032,0,0.233494,0.00813261,-0.148951,0.0813261,0.0813261,-0.0248687,1.0428,0.470084


We just need to make it a matrix

In [48]:
𝚪=Matrix(𝚪ᵦ)


10×10 Matrix{Any}:
  1.22241   0        -0.0100732   …  -0.0710218    0.122241    0.38386
  0         1.22241  -0.0100732      -0.0710218    0.122241    0.38386
  0.325108  0         1.06115        -0.0188888    0.0325108   0.4567
 -0.684548  0         0.00564097      0.0397722   -0.0684548  -0.214962
  0.220814  0         0.0407336      -0.0128293    0.0220814   0.416858
  0.206199  0         0.0138327   …  -0.00198018   0.0206199   0.426595
  0.206199  0         0.0138327      -0.00198018   0.0206199   0.426595
 -0.134465  0         0.00110805      1.00781     -0.0134465  -0.0422246
  0.428032  0         0.233494       -0.0248687    1.0428      0.470084
  0.718619  0         0.185568       -0.0417518    0.0718619   1.28949

# **2. Background knowledge**

## **2.1 Computational LCA**

First, we explain the basics of technosphere and biosphere exchanges.

In [85]:
SMR=getAct("hydrogen production, steam reforming","RoW")
lca(SMR)[end,:]


Row,Exchanges,Climate change-Energy imbalance,Climate change-CO2 Concentration,Ocean acidification-Carbonate ion concentration,Atmospheric aerosol loading-Aerosol Optical Depth (AOD),Freshwater use-Global,Biogeochemical flows-P,Biogeochemical flows-N,Stratospheric ozone depletion-Stratospheric O3 concentration,Land-system change-Global,Biosphere Integrity-Change in biosphere integrity
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
40,TOTAL,3.72281e-12,2.81667e-10,8.6058e-13,7.78278e-15,3.91042e-10,9.38267e-12,1.91231e-12,1.94085e-15,1.4544e-15,2.38269e-12


**Technosphere exchanges**
$$
\boldsymbol{As=f}
\tag{1}
$$


In [62]:
𝐀=Technosphere!().Matrix


23559×23559 SparseMatrixCSC{Float64, Int64} with 291159 stored entries:
⎡⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡏⢸⎤
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡯⢼⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡧⢼⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣯⢼⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣧⢼⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡧⢸⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣽⣷⣺⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣹⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣗⢸⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣷⣇⢼⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡿⡿⢾⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣧⢸⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡧⢸⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⢼⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣯⡯⢹⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣷⢼⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣟⢻⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡏⢹⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⢯⡗⠊⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠛⢯⎦

In [86]:
SMR.key


20436

In [87]:
𝐟 =zeros(𝐀.m)
𝐟[SMR.key]=1 # kgH2/yr


1

In [88]:
𝐬=𝐀\𝐟


23559-element Vector{Float64}:
  5.377890524729087e-7
  3.5062077322241574e-12
  4.855325803316543e-12
  0.0
  0.0002539789356333746
  5.731000073263545e-10
  4.509626323112087e-8
  1.1169672015931043e-6
  0.0
 -0.0
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

**Biosphere exchanges (total inventory)**

$$
\boldsymbol{Bs=BA^{-1}f=g}
\tag{2}
$$


In [89]:
𝐁=Biosphere!().Matrix


4709×23559 SparseMatrixCSC{Float64, Int64} with 437621 stored entries:
⎡⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣷⣿⎤
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎣⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⎦

In [90]:
𝐠 =𝐁*𝐬


4709-element Vector{Float64}:
 5.306669189174679e-15
 2.9004249171139055e-12
 0.0
 0.005478704778945987
 0.0004530910052546953
 0.0
 1.629495207438524e-9
 0.0
 0.0
 0.0
 ⋮
 1.1030998583665938e-11
 0.0
 4.248328108454661e-7
 0.0
 0.00030638420272426114
 2.9479616584594197e-13
 1.9027130478682404e-11
 0.0
 1.0396030498590655e-8

**Impact score calculation**
$$
\boldsymbol{QBs=QBA^{-1}f=h}
\tag{3}
$$

In [91]:
𝐐=Characterisation!().Matrix
𝐡=𝐐*𝐠


10-element Vector{Float64}:
 3.722812863227291e-12
 2.8166669465216066e-10
 8.605802635455069e-13
 7.782779710254727e-15
 3.9104175936896077e-10
 9.382674532739888e-12
 1.912309790513869e-12
 1.9408514089273713e-15
 1.4544031457653483e-15
 2.3826881218181536e-12

**Absolute environmental sustainability assessment calculation**
$$
\boldsymbol{QBs\oslash\omega =QBA^{-1}f\oslash\omega =h\oslash\omega}
\tag{3}
$$

In [92]:
𝐝 = 𝐡 ⊘ 𝛚


10-element Vector{Float64}:
 233.04990891011286
 244.89545921674056
  78.30340997857319
   6.960083176653162
   6.119851423446544
  53.39636563492781
   1.930831123515074
   0.0086784463537237
   0.0036418539753874488
  14.915744361900428

Not very sustainable...


<div style="text-align:center;">
  <img src="./Images/PBI.svg" style="height:400px; display:inline-block;">
</div>


With interactions it's even worse...

In [ ]:
𝐱 = 𝚪*𝐝


10-element Vector{Any}:
  302.14951229513207
  316.62959655100786
  168.94968387197488
 -162.24364370862088
   69.21517065579812
  113.60414569375143
  113.60414569375143
  -33.22776790611082
  128.69611527694732
  208.48742409822725

## **2.2 Life cycle optimisation models**

**Here, we implement the following model**

$$
\min_{\text{s.t.}\ s} \ \boldsymbol{x  = \Gamma d} 
\tag{1}
$$


$$
\boldsymbol{d = \left(\Lambda s\right)\oslash \omega = \left(Q B s\right)\oslash \omega}
\tag{2}
$$

$$
\boldsymbol{A s = f}
\tag{3}
$$

$$
\boldsymbol{s^j \geq 0}
\tag{4}
$$

$$
\boldsymbol{s^{H_2} \leq c^{H_2}}
\tag{5}
$$
$$
\boldsymbol{s^{el} \leq c^{el}}
\tag{6}
$$

<div style="text-align:center;">
  <img src="./Images/rectangularisation.svg" style="height:400px; display:inline-block;">
</div>


What is the interaction model doing?

<div style="text-align:center;">
  <img src="./Images/interactions.svg" style="height:400px; display:inline-block;">
</div>


Okay now let's go to data preparation on activity browser and excel!

# **3.Modelling**


## **3.1 data**

## **3.2 Optimisation**

In [ ]:
model = Model(CPLEX.Optimizer)
set_silent(model)

# Variables and expressions
@variable(model, 𝐬[1:length(processes)])

@expression(model, 𝐝, Λ*𝐬 ⊘ ω)
@expression(model, 𝐱, 𝚪*𝐝)
@expression(model, 𝐱, 𝐝)
@expression(model, φ, 𝐌*𝐬)

@objective(model, Min, 𝐱);
@constraint(model, 𝐬 ≥ 0)
@constraint(model, A * 𝐬 == f)
@constraint(model, 𝐬 ≤ c)

optimize!(model);
if !is_solved_and_feasible(model)
    error("Solver did not find an optimal solution")
end

# solution_summary(model)
DataFrame(hcat([catnames_ticks,value.(𝐝),value.(𝐱)]...),["Boundaries","𝐝", "𝐱"])


In [ ]:
value.(φ)


In [ ]:
value.(𝐬)
